<a href="https://colab.research.google.com/github/erkansengonul/aiworkstr/blob/main/Abnormal_Event_Detection_For_Surveillance_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [3]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Datasets/UCF_Crimes/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/Datasets/UCF_Crimes/Explosion/Explosion008_x264.mp4
/content/drive/MyDrive/Datasets/UCF_Crimes/Explosion/Explosion022_x264.mp4
/content/drive/MyDrive/Datasets/UCF_Crimes/Explosion/Explosion023_x264.mp4


In [4]:
pip install numpngw

In [5]:
class Config:
  BATCH_SIZE = 4
  EPOCHS = 3
  VIDEO_NAME = 'Explosion022_x264'
  VIDEO_DIR = '/content/drive/MyDrive/Datasets/UCF_Crimes/Explosion/'+ VIDEO_NAME +'.mp4'
  #VIDEO_DIR = '/content/drive/MyDrive/Datasets/UCF_Crimes/RoadAccidents/'+ VIDEO_NAME +'.mp4'
  #VIDEO_DIR = '/content/drive/MyDrive/Datasets/UCF_Crimes/Assault/'+ VIDEO_NAME +'.mp4'
  ORIGINAL_TRAIN_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/1-Original/Train/'
  ORIGINAL_TEST_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/1-Original/Test/'
  EIGHTBIT_DEPTH_TRAIN_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/2-8bit_depth/Train/'
  EIGHTBIT_DEPTH_TEST_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/2-8bit_depth/Test/'
  BACKGROUND_REMOVED_TRAIN_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/3-Background_removed/Train/'
  BACKGROUND_REMOVED_TEST_DIR= '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/3-Background_removed/Test/'
  RESIZED_BACK_REMOVED_TRAIN_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/4-Resized_back_removed/Train/'
  RESIZED_BACK_REMOVED_TEST_DIR = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/4-Resized_back_removed/Test/'
  TEST_RESULTS = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/5-Test_results/'  
  MODEL_PATH = '/content/drive/MyDrive/video_anomaly_detection/'+ VIDEO_NAME + '/model.hdf5'
  DATA_PROCESS_METHOD = ''
  TEST_FRAME_COUNT = 0
  TRAIN_FRAME_COUNT = 0

In [6]:
def delete_folder_files(directory):
    #Tüm Output dosyalarını siler
    import os
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            os.remove(os.path.join(dirname, filename))

In [7]:
def release_list(mylist):
   del mylist[:]
   del mylist
    

In [8]:
import cv2
import numpy as np

cap = cv2.VideoCapture(Config.VIDEO_DIR)

if not os.path.exists(Config.ORIGINAL_TRAIN_DIR): os.makedirs(Config.ORIGINAL_TRAIN_DIR)
if not os.path.exists(Config.ORIGINAL_TEST_DIR): os.makedirs(Config.ORIGINAL_TEST_DIR)
if not os.path.exists(Config.EIGHTBIT_DEPTH_TRAIN_DIR): os.makedirs(Config.EIGHTBIT_DEPTH_TRAIN_DIR)
if not os.path.exists(Config.EIGHTBIT_DEPTH_TEST_DIR): os.makedirs(Config.EIGHTBIT_DEPTH_TEST_DIR)
if not os.path.exists(Config.BACKGROUND_REMOVED_TRAIN_DIR): os.makedirs(Config.BACKGROUND_REMOVED_TRAIN_DIR)
if not os.path.exists(Config.BACKGROUND_REMOVED_TEST_DIR): os.makedirs(Config.BACKGROUND_REMOVED_TEST_DIR)
if not os.path.exists(Config.RESIZED_BACK_REMOVED_TRAIN_DIR): os.makedirs(Config.RESIZED_BACK_REMOVED_TRAIN_DIR)
if not os.path.exists(Config.RESIZED_BACK_REMOVED_TEST_DIR): os.makedirs(Config.RESIZED_BACK_REMOVED_TEST_DIR)  
if not os.path.exists(Config.TEST_RESULTS): os.makedirs(Config.TEST_RESULTS)

In [9]:
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
from PIL import Image

def get_clips_by_stride(stride, frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    stride : int
        The distance between two consecutive frames
    frames_list : list
        A list of sorted frames of shape 256 X 256
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , 10 frames each
    """
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

In [10]:
import cv2
import numpy as np
from numpngw import write_png
from PIL import Image

def save_image_8bitdepth(path, file_name, image):
    img = Image.fromarray(image, 'RGB')
    img= img.quantize()
    img.save(path + file_name)

In [11]:
def adaptive_threshold(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 199, 5) 

In [12]:
def preprocess_data(data_process_method, frame_catching_interval, train_set_percentage):
    Config.DATA_PROCESS_METHOD = data_process_method
    #from PIL import Image
    #delete_folder_files('/kaggle/working/')
    #Toplam frame sayısı
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    train_finish_frame_no = int(total_frame * train_set_percentage / 100)

    if data_process_method == 'ABSDIFF':
        #cap.set(cv2.CAP_PROP_POS_FRAMES, 40)
        rval, frame1 = cap.read()
    elif data_process_method == 'MOG2':
        fgbg = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold = 16, detectShadows = False)
    elif data_process_method == 'BLOB':
        detector = cv2.SimpleBlobDetector()  

    original_frames = []
    background_removed_frames = []

    train_frame_count=0
    test_frame_count=0
    for i in range(0,total_frame,frame_catching_interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        original_frames.append(frame)
        
        if data_process_method == 'NONE':
            background_removed_frames.append(frame)
        elif data_process_method == 'ABSDIFF':
            background_removed_frames.append(cv2.erode(cv2.absdiff(frame,frame1), None, 3))
        elif data_process_method == 'MOG2':
            background_removed_frames.append(cv2.erode(fgbg.apply(frame), None, 3))
        elif data_process_method == 'BLOB':
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            keypoints = detector.detect(frame)
            im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0, 0, 255),  
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  
            background_removed_frames.append(im_with_keypoints)

        if i <= train_finish_frame_no:
            cv2.imwrite(Config.ORIGINAL_TRAIN_DIR + str(f'{train_frame_count:04}')+'.png', original_frames[-1])
            save_image_8bitdepth(Config.EIGHTBIT_DEPTH_TRAIN_DIR, str(f'{train_frame_count:04}')+'.png', original_frames[-1])

            if data_process_method == 'NONE':
                save_image_8bitdepth(Config.BACKGROUND_REMOVED_TRAIN_DIR, str(f'{train_frame_count:04}')+'.png', background_removed_frames[-1])
            elif data_process_method == 'ABSDIFF':
                save_image_8bitdepth(Config.BACKGROUND_REMOVED_TRAIN_DIR, str(f'{train_frame_count:04}')+'.png', background_removed_frames[-1])
            elif data_process_method == 'MOG2':
                cv2.imwrite(Config.BACKGROUND_REMOVED_TRAIN_DIR + str(f'{train_frame_count:04}')+'.png', background_removed_frames[-1])
            elif data_process_method == 'BLOB':
                save_image_8bitdepth(Config.BACKGROUND_REMOVED_TRAIN_DIR, str(f'{train_frame_count:04}')+'.png', background_removed_frames[-1])

            train_frame_count=train_frame_count+1
        else:
            cv2.imwrite(Config.ORIGINAL_TEST_DIR + str(f'{test_frame_count:04}')+'.png', original_frames[-1])
            save_image_8bitdepth(Config.EIGHTBIT_DEPTH_TEST_DIR, str(f'{test_frame_count:04}')+'.png', original_frames[-1])
            
            if data_process_method == 'NONE':
                save_image_8bitdepth(Config.BACKGROUND_REMOVED_TEST_DIR, str(f'{test_frame_count:04}')+'.png', background_removed_frames[-1])
            elif data_process_method == 'ABSDIFF':
                save_image_8bitdepth(Config.BACKGROUND_REMOVED_TEST_DIR, str(f'{test_frame_count:04}')+'.png', background_removed_frames[-1])
            elif data_process_method == 'MOG2':
                cv2.imwrite(Config.BACKGROUND_REMOVED_TEST_DIR + str(f'{test_frame_count:04}')+'.png', background_removed_frames[-1])
            elif data_process_method == 'BLOB':
                save_image_8bitdepth(Config.BACKGROUND_REMOVED_TEST_DIR, str(f'{test_frame_count:04}')+'.png', background_removed_frames[-1])

            test_frame_count=test_frame_count+1

    Config.TRAIN_FRAME_COUNT = train_frame_count
    print('Eğitim Sayısı:' + str(train_frame_count))
    Config.TEST_FRAME_COUNT = test_frame_count
    print('Test Sayısı:' + str(test_frame_count))
    #print(len(original_frames))
    #print(len(background_removed_frames))
    #release_list(original_frames)
    #release_list(background_removed_frames)


In [13]:
def get_training_set():
    clips = []
    all_frames = []
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(Config.BACKGROUND_REMOVED_TRAIN_DIR)):
        img_path = join(Config.BACKGROUND_REMOVED_TRAIN_DIR, f)
        img = Image.open(img_path).resize((256, 256))
        img = np.array(img, dtype=np.float32) / 256.0
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        all_frames.append(img)
        cv2.imwrite(Config.RESIZED_BACK_REMOVED_TRAIN_DIR + str(f'{f:4}'), img)

        # get the 10-frames sequences from the list of images after applying data augmentation
    for stride in range(1, 3):
        clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
    return clips

In [14]:
!pip install keras-layer-normalization

  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.15.0-cp37-none-any.whl size=5224 sha256=2285b0c370a77c5e3aa72c7516b628f915b34f5df6e6f9f3541b2e56bf867bb2
  Stored in directory: /root/.cache/pip/wheels/de/ea/db/833c8a9b8326e703e9f8a78c0d4153294e6a1b1f97a1836397
Successfully built keras-layer-normalization


In [15]:
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras_layer_normalization import LayerNormalization
from keras.utils.vis_utils import plot_model

def get_model(reload_model=True):
    """
    Parameters
    ----------
    reload_model : bool
        Load saved model or retrain it
    """
    if not reload_model:
        return load_model(Config.MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
    training_set = get_training_set()
    training_set = np.array(training_set)
    model = Sequential()
    model.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, padding="same"), batch_input_shape=(None, 10, 256, 256, 1)))
    model.add(LayerNormalization())
    model.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, padding="same")))
    model.add(LayerNormalization())
    # # # # #
    model.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    model.add(LayerNormalization())
    model.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
    model.add(LayerNormalization())
    model.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    model.add(LayerNormalization())
    # # # # #
    model.add(TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, padding="same")))
    model.add(LayerNormalization())
    model.add(TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, padding="same")))
    model.add(LayerNormalization())
    model.add(TimeDistributed(Conv2D(1, (11, 11), activation="sigmoid", padding="same")))
    print(model.summary())
    
    myOptimizer = keras.optimizers.Adam(lr=1e-4, decay=1e-5, epsilon=1e-6)
    model.compile(loss='mse', optimizer= myOptimizer)
    model.fit(training_set, training_set,batch_size=Config.BATCH_SIZE, epochs=Config.EPOCHS, shuffle=False)
    model.save(Config.MODEL_PATH)
    plot_model(model, to_file=Config.TEST_RESULTS + 'model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model


In [16]:
def get_test_set():
    sz = Config.TEST_FRAME_COUNT

    #print('Test sayısı: ' + str(sz))
    test = np.zeros(shape=(sz, 256, 256, 1))

    cnt = 0
    for f in sorted(listdir(Config.BACKGROUND_REMOVED_TEST_DIR)):
        img = Image.open(join(Config.BACKGROUND_REMOVED_TEST_DIR, f)).resize((256, 256))
        img = np.array(img, dtype=np.float32) / 256.0
        cv2.imwrite(Config.RESIZED_BACK_REMOVED_TEST_DIR + str(f'{f:4}'), img)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        test[cnt, :, :, 0] = img
        cnt = cnt + 1
    return test

In [17]:
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

sr=[]
def evaluate(reload_model=True):
    model = get_model(reload_model)
    print("got model")
    test = get_test_set()
    print("got test")
    sz = test.shape[0] - 10
    sequences = np.zeros((sz, 10, 256, 256, 1))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((10, 256, 256, 1))
        for j in range(0, 10):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip

    # get the reconstruction cost of all the sequences
    reconstructed_sequences = model.predict(sequences,batch_size=4)
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
    sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
    
    sr = 1.0 - sa
    # plot the regularity scores
   
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    #major_interval = Config.TEST_FRAME_COUNT/6-Config.TEST_FRAME_COUNT%10
    major_interval = round((Config.TEST_FRAME_COUNT/6)-(Config.TEST_FRAME_COUNT/6)%10)
    minor_interval = round(major_interval/5)
    # Major ticks every 20, minor ticks every 5
    major_ticks = np.arange(0, sr.shape[0]+1, major_interval)
    minor_ticks = np.arange(0, sr.shape[0]+1, 10)
    
    ax.set_xticks(major_ticks)
    ax.set_xticks(minor_ticks, minor=True)

    #ax.grid(which='both')
    ax.grid(which='minor', alpha=0.4)
    ax.grid(which='major', alpha=0.8)
    plt.plot(sr)
    plt.ylabel('Düzenlilik Puanı Dp(f)')
    plt.xlabel('Çerçeve f')
    plt.xlim([0, sr.shape[0]])
    
    plt.savefig(Config.TEST_RESULTS + 'Result'+ Config.DATA_PROCESS_METHOD +'.png')
    #plt.show()

In [1]:
#Config.VIDEO_DIR = '/kaggle/input/ucfcrimepart3/Anomaly-Videos-Part-3/RoadAccidents/'+ Config.VIDEO_NAME +'.mp4'

#data_process_method = ['NONE', 'ABSDIFF', 'MOG2']
preprocess_data(data_process_method = 'ABSDIFF', frame_catching_interval = 3, train_set_percentage = 50)

evaluate(reload_model=Ture);

NameError: ignored